In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import train_test_split

# Đọc dữ liệu từ file ratings.csv
ratings = pd.read_csv('ratings.csv')

# Chuyển cột CustomerID thành chuỗi để tránh lỗi khi xử lý
ratings['CustomerID'] = ratings['CustomerID'].astype(str)

# Chuẩn hóa dữ liệu với Surprise Reader
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(ratings[['CustomerID', 'stockCodeTransform', 'rate']], reader)

# Chia dữ liệu thành tập huấn luyện và kiểm tra (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Lớp Item-Based Collaborative Filtering
class ItemBasedWithTopItems(KNNWithMeans):
    def __init__(self, sim_options={'name': 'cosine', 'user_based': False}, *args, **kwargs):
        super().__init__(sim_options=sim_options, *args, **kwargs)

# Tạo đối tượng từ lớp ItemBasedWithTopItems
model = ItemBasedWithTopItems()

# Huấn luyện mô hình trên tập huấn luyện
model.fit(trainset)

# Dự đoán trên tập huấn luyện
train_predictions = model.test(trainset.build_testset())

# Dự đoán trên tập kiểm tra
test_predictions = model.test(testset)

# Tính RMSE cho tập huấn luyện và kiểm tra
train_rmse = accuracy.rmse(train_predictions)
test_rmse = accuracy.rmse(test_predictions)


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8581
RMSE: 0.9711


In [2]:
# Tạo tập dữ liệu đầy đủ 
full_data = data.build_full_trainset()

In [3]:

user_ids = [full_data.to_raw_uid(u) for u in range(full_data.n_users)]  # Người dùng
item_ids = [full_data.to_raw_iid(i) for i in range(full_data.n_items)]  # Sản phẩm

# Tạo ma trận Yhat với đúng thứ tự: hàng là sản phẩm, cột là người dùng
Yhat = np.zeros((len(item_ids), len(user_ids)))  # Khởi tạo ma trận dự đoán

# Điền giá trị dự đoán vào ma trận
for item_idx, item_id in enumerate(item_ids):  # Sản phẩm làm hàng
    for user_idx, user_id in enumerate(user_ids):  # Người dùng làm cột
        est = model.predict(user_id, item_id).est
        Yhat[item_idx, user_idx] = est

# Làm tròn giá trị đến chữ số thập phân thứ nhất
Yhat = np.round(Yhat, 1)
# Lưu ma trận Yhat vào file numpy
np.save('Yhat_IB.npy', Yhat)

# Để tải lại ma trận Yhat sau này
Yhat_loaded = np.load('Yhat_IB.npy')
print(Yhat_loaded)
Yhat_loaded.shape[0]

[[4.3 4.  4.  ... 3.4 1.5 4.3]
 [4.1 3.4 3.5 ... 2.8 1.  3.8]
 [5.  5.  5.  ... 4.7 3.  5. ]
 ...
 [3.  3.  3.8 ... 3.  1.  3. ]
 [1.9 1.4 2.  ... 1.  1.  2.1]
 [2.8 3.1 2.  ... 2.1 1.  3.3]]


3642

In [4]:

import pickle

# Lưu mô hình vào tệp
with open('item_based_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Tải mô hình đã lưu
with open('item_based_model.pkl', 'rb') as file:
    model = pickle.load(file)
